In [1]:
import importlib
import copy
# --------------------
from sklearn.utils import gen_batches
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error as RMSE
import numpy as np
from numpy.linalg import norm,inv,matrix_rank

import h5py
from sklearn.cross_decomposition import PLSRegression

# import warnings
# warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()

In [2]:
with h5py.File('./data/TW_PM25.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)
        
    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]
 

877


In [4]:
def RollingCV(tscv,X):
    cv = tscv.split(X)
    (train_index, test_index) = next(cv)
    yield(
        train_index,
        test_index
    )
    test_size = len(test_index)

    for (train_index, test_index) in (cv):
        yield(
            train_index[-test_size:],
            test_index
        )


In [5]:
print(X_train.shape)
print(Y_train.shape)
n_fold = 17
test_size=30
tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)


for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break
    
print("="*40)
for i, (train_index, test_index) in enumerate(RollingCV(tscv,X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break    

(877, 1800)
(877, 1800)
Fold 0:
  Train: len=367
  Test:  len=30
Fold 1:
  Train: len=397
  Test:  len=30
Fold 2:
  Train: len=427
  Test:  len=30
Fold 3:
  Train: len=457
  Test:  len=30
Fold 4:
  Train: len=487
  Test:  len=30
  Train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187

In [ ]:
import Code.SIMPLS
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

n_comp_lst=list(range(1,21))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(SIMPLS(),  parameters,
                   cv=tscv, verbose=2,  
                   scoring='neg_root_mean_squared_error')
print(clf)
tim.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tim.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))


In [89]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

n_comp_list=list(range(1,21))
print("n_comp:", n_comp_list)
score_all = np.zeros((n_fold, len(n_comp_list)))

n_comp_max=20
PLS = ISIMPLS(n_components=n_comp_max)

tim_tot.tic()
tim.tic()
for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    y_true = Y_train[test_index]
    for j,n_comp in enumerate(n_comp_list):
        y_pred = PLS.predict(X_train[test_index], n_comp)
        score_all[i,j] = RMSE(y_true, y_pred)
    if (i%5==0): 
        print(f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()

scores = score_all.mean(axis=0)
best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {n_comp_list[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")
print(scores)

n_comp: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


fold: 01, elapsed time: 3.1s


fold: 06, elapsed time: 12.8s


fold: 11, elapsed time: 14.0s


fold: 16, elapsed time: 12.5s


[7.76705994 7.50395476 7.36823013 7.31293782 7.31652096 7.29054187
 7.2979135  7.2500481  7.23974823 7.16567089 7.14763537 7.19020176
 7.19894527 7.21953187 7.24954174 7.27576464 7.32228498 7.33506288
 7.33961495 7.38431746]

best parameter: 11; score: 7.1476354e+00; total time=44.9s


In [96]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

PLS_list = []
n_comp_max=20
PLS = ISIMPLS(n_components=n_comp_max)
tim_tot.tic()
tim.tic()
for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    PLS_list.append(copy.deepcopy(PLS))
    if (i%5==0): 
        print(f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()
print(f"total time={tim_tot.tocvalue():.1f}s")

fold: 01, elapsed time: 2.4s


fold: 06, elapsed time: 11.5s


fold: 11, elapsed time: 11.1s


fold: 16, elapsed time: 10.0s


total time=37.0s


In [97]:
def Comp_Model_Score(model, n_comp):
    scores = np.zeros((n_fold,))
    for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
        y_true = Y_train[test_index]
        y_pred = model[i].predict(X_train[test_index], n_comp)
        scores[i] = RMSE(y_true, y_pred)
    return np.mean(scores)


In [107]:
tim_tot.tic()

left = 1
right = n_comp_max
path=[]
yl = Comp_Model_Score(PLS_list, left)
yr = Comp_Model_Score(PLS_list, right)
path.append((left, yl))
# path.append((right, yr))

while (right-left != 1):
    m = (yr-yl)/(right-left)
    if m > 0:
        right = left + (right - left) // 2
        yr = Comp_Model_Score(PLS_list, right)
        path.append((right, yr))
    else:
        left = left + (right - left) // 2
        yl = Comp_Model_Score(PLS_list, left)
        path.append((left, yl))


if yl < yr:
    best_ind = left-1
    best_score = yl
else:
    best_ind = right-1
    best_score = yr

print(f"best parameter: {n_comp_list[best_ind]}; score: {best_score:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")

best parameter: 11; score: 7.1476354e+00; total time=2.7s
